In [ ]:
!pip install sagemaker
!pip install boto3
!pip install pandas
!pip install pyodbc sqlalchemy

# install ODBC
!sudo su -c "curl https://packages.microsoft.com/config/rhel/7/prod.repo > /etc/yum.repos.d/mssql-release.repo"
!sudo yum remove unixODBC-utf16 -y
!sudo ACCEPT_EULA=Y yum install -y msodbcsql17

# Install unixODBC and pyodbc
!sudo yum install -y unixODBC unixODBC-devel gcc gcc-c++ 
!pip install pyodbc

In [19]:
import sagemaker
import boto3
import pandas as pd
from io import StringIO

sagemaker.get_execution_role()

'arn:aws:iam::905418122102:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole'

In [3]:
bucket = 'sagemaker-agd'
key = 'pet_adoption_center.csv'
s3 = boto3.client('s3')
response = s3.get_object(Bucket=bucket, Key=key)
df = pd.read_csv(StringIO(response['Body'].read().decode('utf-8')))


In [4]:
df.head()


,pet_id,pet_name,species,breed,age_years,gender,color,arrival_date,adopted,adoption_date,adopter_id,adopter_name,adopter_age,adopter_city,adopter_previous_pets
0,P0001,Sandra,Dog,Yorkshire Terrier,2,Female,White,2024-10-10,True,2025-03-11,A0001,Megan Rogers,56,Mcgeestad,1
1,P0002,Brian,Cat,Tonkinese,1,Female,Golden,2023-09-09,True,2024-02-10,A0002,Jonathan Hawkins,18,Geneshire,2
2,P0003,Timothy,Bird,Amazon Parrot,4,Male,Spotted,2023-10-03,False,2023-10-17,NaN,NaN,-1,NaN,0
3,P0004,Karl,Dog,German Shepherd,1,Female,White,2024-07-28,True,2024-08-12,A0004,Natalie Rivera,51,North Richard,3
4,P0005,Sarah,Rabbit,Dutch,1,Female,Spotted,2024-02-21,False,2024-02-26,NaN,NaN,-1,NaN,0


In [5]:
import json
secret_name = 'rds!db-59c6932b-113f-421a-ae20-51a26fc48d8d'
region_name = 'us-east-1'

session = boto3.session.Session()
client = session.client(service_name='secretsmanager', region_name=region_name)

response = client.get_secret_value(SecretId=secret_name)
secret = json.loads(response['SecretString'])

username = secret['username']
password = secret['password']

print(username)
print(password)

admin
SzH~0(~GCgajymUDtn??DcKB*9B5


In [14]:
import sqlalchemy
from sqlalchemy import text

# connect to DB
endpoint = 'agd-sqlserver-express.cg1ugiyqy9d8.us-east-1.rds.amazonaws.com'
database = 'mydata'

conn_str = (
    f"mssql+pyodbc://{username}:{password}@{endpoint}:1433/{database}"
    f"?driver=ODBC+Driver+17+for+SQL+Server"
)

engine = sqlalchemy.create_engine(conn_str)


In [17]:
# export data to DB
df.to_sql('pet_adoption_center', engine, if_exists='replace', index=False)


with engine.connect() as conn:
    result = conn.execute(text("SELECT COUNT(*) FROM pet_adoption_center"))
    count = result.scalar()
    print("Row count:", count)

Row count: 200


In [18]:
# export to s3

local_path = '/tmp/exportdata.csv'
df.to_csv(local_path, index=False)

s3 = boto3.client('s3')
bucket = 'sagemaker-agd'
s3_path = 'exportdata.csv'
s3.upload_file(local_path, bucket, s3_path)
print(f"Upladed to s3://{bucket}/{s3_path}")

Upladed to s3://sagemaker-agd/exportdata.csv


In [ ]:
# import from DB
table_name = 'pet_adoption_center'

# Simple SELECT query
query = f"SELECT * FROM {table_name}"

# Read into DataFrame
df = pd.read_sql(query, con=engine)

# Preview
df.head()